# Japanese Phrases Generator Project

A multi-agent system for generating Japanese phrases across different verb tenses.

## System Overview
This notebook implements an intelligent system that leverages multiple AI agents to generate grammatically correct Japanese sentences. The system focuses on verb conjugation and tense variations.

### Key Components

#### Agents
- **Chatbot Agent**: Manages user interactions and maintains system state
- **Sentence Expert Agent**: Specialized in Japanese sentence construction and grammar rules

#### Features
- Dynamic phrase generation
- Support for multiple verb tenses
- State management and persistence
- Modular and extensible architecture

#### Project Structure
- Tools and utilities for agent interaction
- State management system
- Agent implementation modules

# IMPORTANT!
The app built in this notebook takes user input using a text box (Python's input). These are commented-out to ensure that you can use the Run all feature without interruption. Keep an eye out for the steps where you need to uncomment the .invoke(...) calls in order to interact with the app.


# Get set up
Start by installing and importing the LangGraph SDK and LangChain support for the Gemini API.
We will use pydantic do define structured output,
The dango package will be used to check if a word is a verb, and to tokenize the generated phrase.


In [16]:
# Remove conflicting packages from the Kaggle base environment.
# !pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai

# Install langgraph and the packages used in this lab.
!pip install langchain-core==0.3.52 langgraph==0.3.29 typing_extensions==4.13.2 dango==0.0.1 langchain-google-genai==2.1.2 pydantic==2.11.3

# Set up your API key
The GOOGLE_API_KEY environment variable can be set to automatically configure the underlying API. This works for both the official Gemini Python SDK and for LangChain/LangGraph.

To run the following cell, your API key must be stored it in a Kaggle secret named GOOGLE_API_KEY.

If you don't already have an API key, you can grab one from AI Studio. You can find detailed instructions in the docs.

To make the key available through Kaggle secrets, choose Secrets from the Add-ons menu and follow the instructions to add your key or enable it for this notebook.

In [17]:
## Uncomment on a Keggle environment
#import os
#from kaggle_secrets import UserSecretsClient

#GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
#os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Tools for the chatbot

This set of tools forms the core of the user interaction for collecting the necessary ingredients – Japanese verbs and grammatical tenses – before the system can generate phrases.
Think of it as the agent's toolkit for managing a virtual "notebook" where user selections are recorded.


### List of supported tenses

First the chatbot agent can call the `get_verb_tenses()` method. It works as a "Menu" from which the user should pick the tense.

The formatting of the Non-Past Tense and Past Tense parts informs the chatbot that the user should pick between one of the choices and not a generic "past tense".

In [18]:
from langchain_core.tools import tool


@tool
def get_verb_tenses() -> str:
    """Get a list of all the verb tenses."""

    return """
    GRAMMAR:
    Non-Past Tense:
    Non-Past Affirmative Plain
    Non-Past Affirmative Polite
    Non-Past Negative Plain
    Non-Past Negative Polite

    Past Tense:
    Past Affirmative Plain
    Past Affirmative Polite
    Past Negative Plain
    Past Negative Polite

    Te Form
    Desiderative
    Volitional
    Exhortative
    Imperative
    Request
    Potential
    Passive
    Causative
    Conditional
    """

### Verifying User Input: The `is_verb` Tool

A critical function of the chatbot is to ensure the words provided by the user are suitable for phrase generation – specifically, that they are Japanese verbs. While the powerful language model underlying the chatbot *could* attempt this check, relying solely on it carries a risk of occasional errors or "hallucinations."

To guarantee accuracy and predictability, the system employs a dedicated tool: `is_verb`. When the user suggests a word, the chatbot calls this tool. `is_verb` then uses the specialized `dango` Japanese language library to definitively determine if the word's part of speech is actually a verb. This approach provides a reliable, rule-based verification, ensuring only valid verbs are added to the notebook for phrase generation.

In [19]:
from dango.word import PartOfSpeech
from dango import dango


@tool
def is_verb(word: str):
    """Call to verify that a japanese word is a verb."""
    tokens = dango.tokenize(word)
    if len(tokens) != 1:
        raise ValueError("Input must be a single word.")
    only_word = tokens[0]
    return only_word.part_of_speech == PartOfSpeech.VERB

## State Management

The chatbot agent maintains a state object that tracks the conversation and the current state of the system.

The state object is a dictionary that contains the following keys:

* `messages`: A list of messages sent by the user and received by the chatbot.
* `verbs`: A list of verbs selected by the user.
* `tenses`: A list of tenses selected by the user.
* `generated_phrases`: A list of the sentences generated by the sentence expert agent.

In [20]:
from typing_extensions import TypedDict, Annotated

from langgraph.graph import add_messages


class PhraseGeneratorState(TypedDict):
    """State representing the conjugation of a verb."""

    # The chat conversation. This preserves the conversation history
    # between nodes. The add_messages annotation defines how this
    # state key should be updated (in this case, it appends messages
    # to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

    # The verbs to use to generate the sentences for
    verbs: list[str]

    # The tense to generate the sentences for
    tenses: list[str]

    # Flag indicating whether the sentence generation is complete
    finished: bool

    generated_phrases: list[str]

### State Management Tools

The notebook uses state management tools to track conversation and system state during interactions:

- `add_verb()`: Adds new verbs to the state
- `clear_verbs()`: Removes all verbs from state
- `add_tense()`: Adds grammatical tenses to state 
- `clear_tenses()`: Removes all tenses from state
- `get_notebook()`: Retrieves current state contents
- `confirm_generation()`: Validates state before phrase generation

Because LangChain doesn't allow tools to update the state, the void tools allows the LLM to signal wanting to use a certain function,
but the state update is effectively made by the collector_node.


In [21]:
@tool
def add_verb(verb: str) -> str:
    """Adds the specified verb to the notebook.

    Returns:
        The updated verb list.
    """

@tool
def clear_verbs():
    """Removes the selected verb from the notebook."""

@tool
def add_tense(tense: str) -> str:
    """Adds the specified tense to the notebook.

    Returns:
    The updated verb list.
    """

@tool
def clear_tenses():
    """Removes all tenses from the notebook."""

@tool
def get_notebook():
    """Returns the current elements in the notebook."""

@tool
def confirm_generation():
    """Confirms the generation of sentences."""

def collector_node(state: PhraseGeneratorState) -> PhraseGeneratorState:
    """The collector node. This is where the phrase generator state is manipulated."""
    tool_msg = state.get("messages", [])[-1]
    verbs = state.get("verbs", [])
    tenses = state.get("tenses", [])
    outbound_msgs = []
    generation_complete = False

    for tool_call in tool_msg.tool_calls:
        if tool_call["name"] == "add_verb":
            verbs.append(tool_call["args"]["verb"])
            response = "\n".join(verbs)
        elif tool_call["name"] == "clear_verb":
            verbs.clear()
        elif tool_call["name"] == "add_tense":
            tenses.append(tool_call["args"]["tense"])
            response = "\n".join(tenses)
        elif tool_call["name"] == "clear_tenses":
            tenses.clear()
        elif tool_call["name"] == "get_notebook":
            response = "Verbs:\n" + "\n".join(verbs) + "\n" + "Tenses:\n" + "\n".join(tenses)
        elif tool_call["name"] == "confirm_generation":
            print("Your notebook:")
            print("Verbs:")
            if not verbs:
                print("   (no verbs)")
            for verb in verbs:
                print(f"   {verb}")
            print("Tenses:")
            if not tenses:
                print("   (no tenses)")
            for tense in tenses:
                print(f"   {tense}")
            response = input("Is this correct? (y/n): ")
        else:
            raise NotImplementedError(f'Unknown tool call: {tool_call["name"]}')

        outbound_msgs.append(
            ToolMessage(
                content=response,
                name=tool_call["name"],
                tool_call_id=tool_call["id"]
            )
        )

    state = {
        "messages": outbound_msgs,
        "verbs": verbs,
        "finished": False,
        "tenses": tenses,
        "generated_phrases": []
    }

    return state

# Sentence Expert Tools: Leveraging Generative AI

Unlike the tools designed for chatbot interactions, our Sentence Expert tools harness the power of generative AI for creating nuanced and contextually relevant sentences.

To ensure structured and analyzable output, we define a `PhraseResponse` model. This allows us to consistently receive and process the generated sentences.

Furthermore, we employ Few-Shot Prompting with detailed system instructions. This technique provides the generative model with examples, guiding it to produce the desired style and quality of sentences.

In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field


class PhraseResponse(BaseModel):
    """Response model for Japanese phrase generation"""
    verb: str = Field(description="The Japanese verb used in phrase generation")
    tense: str = Field(description="The grammatical tense used in phrase generation")
    sentence: str = Field(description="The generated sentence")

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
llm = model.with_structured_output(PhraseResponse)

GENERATOR_SYSINT = (
    "system",  # 'system' indicates the message is a system instruction.
    "You are a Japanese Sentence Generator."
    "Given a verb and a tense, you generate a simple sentence that must contain the given verb in the given tense."
    "You cannot do anything else."
    "The sentence should be simple, with words matching the level of difficulty of the verb."
    "You should not generate sentences that contains only the verb."
    "\n\n"
    "Here you have some examples:"
    ""
    "example_user: Generate a Japanese sentence using the verb 見る in volitive tense."
    "example_assistant: 映画を見に行こう。"
    ""
       ""
    "example_user: Generate a Japanese sentence using the verb 写す in past negative polite tense."
    "example_assistant: 写真を写しませんでした。"
    ""
    ""
    "example_user: Generate a Japanese sentence using the verb 食べる in conditional tense."
    "example_assistant: 食べたら、元気になります。"
    ""

    "\n\n"
    "If any of the tools are unavailable, you can break the fourth wall and tell the user that "
    "they have not implemented them yet and should keep reading to do so.",
)

Following the established pattern, we create a specific node to streamline the generation process for each unique combination of verb and tense. This focused approach ensures efficient and accurate sentence creation.

In [23]:
from langchain_core.messages import ToolMessage


@tool
def generate_sentences():
    """Generate sentences based on the given verb and tense."""

def tokenize_sentence(sentence: str):
    """Tokenize a sentence into words."""

    tokens =  dango.tokenize(sentence)
    result = []
    for token in tokens:
        result.append({
            "dictionary_form": token.dictionary_form,
            "dictionary_form_reading": token.dictionary_form_reading,
            "part_of_speech": token.part_of_speech.value,
            "surface": token.surface,
            "surface_reading": token.surface_reading,
        })
    return result

def generator_node(state: PhraseGeneratorState) -> PhraseGeneratorState:
    """The generator node. This is where the phrase generator state is manipulated."""
    tool_msg = state.get("messages", [])[-1]
    verbs = state.get("verbs", [])
    tenses = state.get("tenses", [])
    outbound_msgs = []
    generated_sentences = []
    generation_complete = False

    for tool_call in tool_msg.tool_calls:
        if tool_call["name"] == "generate_sentences":
            for verb in verbs:
                for tense in tenses:
                    prompt = f"Generate a Japanese sentence using the verb '{verb}' in {tense} tense."
                    response = llm.invoke([GENERATOR_SYSINT] + [("user", prompt)])
                    generated_sentences.append({
                        "verb": response.verb,
                        "tense": response.tense,
                        "sentence": response.sentence,
                        "tokenized_sentence": tokenize_sentence(response.sentence)
                    })

            if not generated_sentences:
                response = "(no sentence generated)"
            else:
                response = "Generated sentences:\n"
                for sentence in generated_sentences:
                    response += f"- {sentence['verb']} ({sentence['tense']}): {sentence['sentence']}\n\n"
        else:
            raise NotImplementedError(f'Unknown tool call: {tool_call["name"]}')

        print(response)

        outbound_msgs.append(
            ToolMessage(
                content=response,
                name=tool_call["name"],
                tool_call_id=tool_call["id"]
            )
        )

    state = {
        "messages": outbound_msgs,
        "verbs": verbs,
        "finished": generation_complete,
        "tenses": tenses,
        "generated_phrases": generated_sentences
    }

    return state

# Handoff tools

Finally the tools that will be used by the agents to transferring conversation to each other.

In [24]:
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool, InjectedToolCallId
from langgraph.prebuilt import InjectedState
from langgraph.types import Command
from typing_extensions import Annotated

@tool
def transfer_to_sentence_expert(state: Annotated[dict, InjectedState],
                                tool_call_id: Annotated[str, InjectedToolCallId]):
    """Transfer the conversation to the sentence expert."""

    state['messages'].append(
        ToolMessage(
            content="Transferring conversation to sentence expert.",
            name="transfer_to_sentence_expert",
            tool_call_id=tool_call_id
        )
    )

    return Command(
        goto="sentence_expert",
        graph=Command.PARENT,
        update=state
    )

@tool
def transfer_to_chatbot(state: Annotated[dict, InjectedState],
                                tool_call_id: Annotated[str, InjectedToolCallId]):
    """Transfer the conversation to the sentence expert."""

    state['messages'].append(
        ToolMessage(
            content="Transferring conversation to chatbot.",
            name="transfer_to_chatbot",
            tool_call_id=tool_call_id
        )
    )
    return Command(
        goto="chatbot",
        graph=Command.PARENT,
        update=state
    )

# Agents Definitions

We start defining the chatbot agent:

Defines a stateful chatbot for generating phrases based on user-selected verbs and tenses.
It uses Google's generative AI model and incorporates tools for interacting with the user
(e.g., adding/clearing verbs and tenses, confirming generation) and external functionalities
(e.g., checking if a word is a verb, getting verb tenses, transferring to a sentence expert).

The chatbot's behavior is managed by a state graph, routing user input and model responses
through different nodes: the chatbot itself, a human interaction node, and a tool execution node.

Key functionalities include:
- Engaging in conversation about verb tenses and meanings.
- Allowing users to build a "notebook" of verbs and tenses.
- Validating user input (verifying verbs and tenses).
- Confirming the generation request with the user before proceeding.
- Potentially delegating phrase generation to a "sentence expert" tool.

The chatbot starts with a welcome message and continues the conversation until the user quits.

In [25]:
from langchain_core.messages import AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from typing_extensions import Literal

CHATBOT_SYSINT = (
    "system",  # 'system' indicates the message is a system instruction.
    "You are a Chatbot of an interactive phrase generator system. You can talk in English."
    "A human will talk to you about the different tenses that a verb can have and you will answer any questions about verb tenses "
    "(and only about verb tenses - no off-topic discussion, but you can chat about the verb meaning). "
    "The user will choose verbs to generate phrases for, and the tenses he wants. "
    "After that you will generate phrases for those verbs in that tenses."
    "You cannot generate phrases if you do not have at least one verb and one tense."
    "\n\n"
    "Add the chosen verb to the user notebook with add_verb, and reset the verbs with clear_verb."
    "Add tenses to the user notebook with add_tense, and reset the tenses with clear_tenses. "
    "To see the contents of the notebook so far, call get_notebook (this is shown to you, not the user) "
    "Always confirm_generation with the user (double-check) before calling generate_phrases. Calling confirm_generation will "
    "display the selected items to the user and returns their response to seeing the list. Their response may contain modifications. "
    "Always verify and respond with tenses from the GRAMMAR before adding them to the notebook. "
    "Always verify for each chosen word if it's a verb with is_verb."
    "You can only generate phrases starting from verbs. "
    "Once the user has finished composing its notebook, Call confirm_generation to ensure it is correct then make "
    "any necessary updates."
    "You can ask the sentence expert for help with generating phrases."
    "\n\n"
    "Do not mention the tools name in responses."
    "If any of the tools are unavailable, you can break the fourth wall and tell the user that "
    "they have not implemented them yet and should keep reading to do so.",
)

WELCOME_MSG = "Welcome to the PhraseGenerator dojo. Type `q` to quit. What phrases can I generate today?"

tools = [get_verb_tenses, is_verb, transfer_to_sentence_expert]
collection_tools = [add_verb, clear_verbs, add_tense, clear_tenses, get_notebook, confirm_generation]


def make_chatbot(llm: ChatGoogleGenerativeAI):
    agent_tools = tools + collection_tools
    llm_with_tools = llm.bind_tools(agent_tools)
    agent_tools_node = ToolNode(agent_tools)

    def maybe_route_to_tools(state: PhraseGeneratorState) -> str:
        """Route between chat and tool nodes if a tool call is made."""
        if not (msgs := state.get("messages", [])):
            raise ValueError(f"No messages found when parsing state: {state}")

        msg = msgs[-1]

        if state.get("finished", False):
            # When an order is placed, exit the app. The system instruction indicates
            # that the chatbot should say thanks and goodbye at this point, so we can exit
            # cleanly.
            return END

        elif hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
            # Route to `tools` node for any automated tool calls first.
            if msg.tool_calls[-1]["name"] in ToolNode(collection_tools).tools_by_name.keys():
                return "collector"
            else:
                return "tools"

        else:
            return "human"

    def chatbot(state: PhraseGeneratorState) -> PhraseGeneratorState:
        """The chatbot itself. A wrapper around the model's own chat interface."""

        if state["messages"]:
            # If there are messages, continue the conversation with the Gemini model.
            new_output = llm_with_tools.invoke([CHATBOT_SYSINT] + state["messages"])
        else:
            # If there are no messages, start with the welcome message.
            new_output = AIMessage(content=WELCOME_MSG)

        return state | {"messages": [new_output]}

    def human_node(state: PhraseGeneratorState) -> PhraseGeneratorState:
        """Display the last model message to the user, and receive the user's input."""
        last_msg = state["messages"][-1]
        print("Model:", last_msg.content)

        user_input = input("User: ")

        # If it looks like the user is trying to quit, flag the conversation
        # as over.
        if user_input in {"q", "quit", "exit", "goodbye"}:
            state["finished"] = True

        return state | {"messages": [("user", user_input)]}

    def maybe_exit_human_node(state: PhraseGeneratorState) -> Literal["chatbot", "__end__"]:
        """Route to the chatbot, unless it looks like the user is exiting."""
        if state.get("finished", False):
            return END
        else:
            return "chatbot"

    # Start building a new graph.
    graph_builder = StateGraph(PhraseGeneratorState)

    # Add the chatbot and human nodes to the app graph.
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("human", human_node)
    graph_builder.add_node("tools", agent_tools_node)
    graph_builder.add_node("collector", collector_node)

    # Chatbot may go to tools, or human.
    graph_builder.add_conditional_edges("chatbot", maybe_route_to_tools)
    # Human may go back to chatbot, or exit.
    graph_builder.add_conditional_edges("human", maybe_exit_human_node)

    # Tools always route back to chat afterwards.
    graph_builder.add_edge("tools", "chatbot")
    graph_builder.add_edge("collector", "chatbot")

    graph_builder.add_edge(START, "chatbot")

    return graph_builder.compile()


Defines a "Sentence Expert" chatbot focused on generating sentences based on user requests.
It utilizes Google's generative AI model and has access to specific tools:
- `generate_sentences`: The primary tool for creating the desired phrases.
- `transfer_to_chatbot`: A tool to hand the conversation back to the main chatbot
                         after sentence generation is complete.

The Sentence Expert operates using a state graph to manage the flow of conversation
and tool calls. It includes nodes for:
- `expert`: The core logic of the sentence expert, interacting with the LLM.
- `human`: Handling user input and displaying the model's responses.
- `tools`: Executing general utility tools (in this case, only `transfer_to_chatbot`).
- `generator`: Specifically executing the `generate_sentences` tool.

The conversation flow involves the expert receiving instructions, potentially calling
`generate_sentences`, and then using `transfer_to_chatbot` to return control to the
main chatbot. The system instruction guides the expert to primarily focus on sentence
generation and then hand off the conversation.

In [26]:
from typing import Literal

from langchain_core.messages import SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.constants import START, END
from langgraph.prebuilt import ToolNode

SENTENCE_EXPERT_SYSINT = """
    You are a Sentence Expert.
    To generate sentences, you can call generate_sentences
    After the generation, you should transfer the conversation to the chatbot.
    \n\n
    If any of the tools are unavailable, you can break the fourth wall and tell the user that
    they have not implemented them yet and should keep reading to do so.
    """

tools = [transfer_to_chatbot]
generator_tools = [generate_sentences]

def make_sentence_expert(llm: ChatGoogleGenerativeAI):
    agent_tools = tools + generator_tools
    llm_with_tools = llm.bind_tools(agent_tools)
    agent_tools_node = ToolNode(agent_tools)

    def maybe_route_to_tools(state: PhraseGeneratorState) -> str:
        """Route between chat and tool nodes if a tool call is made."""
        if not (msgs := state.get("messages", [])):
            raise ValueError(f"No messages found when parsing state: {state}")

        msg = msgs[-1]

        if state.get("finished", False):
            # When an order is placed, exit the app. The system instruction indicates
            # that the chatbot should say thanks and goodbye at this point, so we can exit
            # cleanly.
            return END

        elif hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
            # Route to `tools` node for any automated tool calls first.
            if msg.tool_calls[-1]["name"] in ToolNode(generator_tools).tools_by_name.keys():
                return "generator"
            else:
                return "tools"

        else:
            return "human"

    def expert(state: PhraseGeneratorState) -> PhraseGeneratorState:
        """The chatbot itself. A wrapper around the model's own chat interface."""

        system_message = SystemMessage(content=SENTENCE_EXPERT_SYSINT)
        new_output = llm_with_tools.invoke([system_message] + state["messages"])

        return state | {"messages": [new_output]}

    def human_node(state: PhraseGeneratorState) -> PhraseGeneratorState:
        """Display the last model message to the user, and receive the user's input."""
        last_msg = state["messages"][-1]
        print("Model:", last_msg.content)

        user_input = input("User: ")

        # If it looks like the user is trying to quit, flag the conversation
        # as over.
        if user_input in {"q", "quit", "exit", "goodbye"}:
            state["finished"] = True

        return state | {"messages": [("user", user_input)]}

    def maybe_exit_human_node(state: PhraseGeneratorState) -> Literal["expert", "__end__"]:
        """Route to the chatbot, unless it looks like the user is exiting."""
        if state.get("finished", False):
            return END
        else:
            return "expert"

    # Start building a new graph.
    graph_builder = StateGraph(PhraseGeneratorState)

    # Add the chatbot and human nodes to the app graph.
    graph_builder.add_node("expert", expert)
    graph_builder.add_node("human", human_node)
    graph_builder.add_node("tools", agent_tools_node)
    graph_builder.add_node("generator", generator_node)

    # Chatbot may go to tools, or human.
    graph_builder.add_conditional_edges("expert", maybe_route_to_tools)
    # Human may go back to chatbot, or exit.
    graph_builder.add_conditional_edges("human", maybe_exit_human_node)

    # Tools always route back to chat afterwards.
    graph_builder.add_edge("tools", "expert")
    graph_builder.add_edge("generator", "expert")

    graph_builder.add_edge(START, "expert")

    return graph_builder.compile()

# Code to run the agents

This code sets up the main workflow for the phrase generation system.
It initializes two separate chatbots using Google's Gemini Flash model:
- `chatbot`: The primary interactive bot for discussing verb tenses and collecting
             user preferences for verbs and tenses.
- `sentence_expert`: A specialized bot dedicated to generating the actual phrases
                     based on the information gathered by the main chatbot.

A LangGraph `StateGraph` is created to define the flow between these two chatbots.
Currently, it's a simple graph with:
- A "chatbot" node running the main chatbot logic.
- A "sentence_expert" node running the sentence generation expert logic.
- An edge connecting the start of the graph to the "chatbot", meaning the
  interaction begins with the main chatbot.

The graph is compiled into an executable object.

Finally, the code invokes the graph with an initial empty message list, starting
the conversation with the main chatbot. A recursion limit is set for the graph execution.

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.constants import START
from langgraph.graph import StateGraph

chatbot = make_chatbot(ChatGoogleGenerativeAI(model="gemini-2.0-flash"))
sentence_expert = make_sentence_expert(ChatGoogleGenerativeAI(model="gemini-2.0-flash"))

builder = StateGraph(PhraseGeneratorState)
builder.add_node("chatbot", chatbot)
builder.add_node("sentence_expert", sentence_expert)
builder.add_edge(START, "chatbot")
graph = builder.compile()

config = {"recursion_limit": 100}

# Uncomment this line to execute the graph:
state = graph.invoke({"messages": []}, config)

# Example conversation (only human parts)
# - What tenses do you support?
# - Can you generate a sentence with the verb 食べる at the imperative tense?
# - Can you add verbs 食べる、見る and 写す?
# - Can you add verbs 走る and 今日?
# - What there is my notebook?
# - add Request, volitional, past negative polite and conditional
# - add non past tense

Model: Welcome to the PhraseGenerator dojo. Type `q` to quit. What phrases can I generate today?
Your notebook:
Verbs:
   食べる
Tenses:
   imperative
Model: I understand that you want me to generate a phrase with the verb 食べる at the imperative tense. Is that correct?
Model: OK. Please wait while I generate the phrase.
Model: If you want to generate more phrases, tell me the verb and the tense, or type `q` to quit.


KeyboardInterrupt: Interrupted by user